# 延迟执行 

MXNet使用延迟执行来提升系统性能。绝大情况下我们不用知道它的存在
延迟执行是指命令可以等到它的结果真正的需要的时候再执行。我们来看一个例子：

In [1]:
a=1+1
# some other things 
print(a) 

2



第一句对`a`赋值，再执行一些其指令后打印`a`的结果。因为这里我们可能很久以后才用`a`的值，所以我们可以把它的执行延迟到后面。这样的主要好处是在执行之前系统可以看到后面指令，从而有更多机会来对程序进行优化。例如如果`a`在被使用前被重新赋值了，那么我们可以不需要真正执行第一条语句。

在MXNet里，我们把用户打交道的部分叫做前端。例如这个教程里我们一直在使用Python前端写代码。除了Python外，MXNet还支持其他例如Scala，R，C++的前端。不管使用什么前端，MXNet的程序执行主要都在C++后端。前端只是把程序传给后端。后端有自己的线程来不断的收集任务，构造计算图，优化，并执行。本章我们介绍后端优化之一：延迟执行。

考虑下图的样例，我们在前端调用四条语句，它们被后端的线程分析依赖并构建成计算图。

![](../img/frontend-backend.svg)

在延迟执行中，前端执行前三个语句的时候，它仅仅是把任务放进后端的队列里就返回了。当在需要打印结果时，前端会等待后端线程把`c`的结果计算完。

这个设计的一个好处是前端，就是Python线程，不需要做实际计算工作，从而不管Python的性能如何，它对整个程序的影响会很小。只需要C++后端足够高效，那么不管前端语言性能如何，都可以提供一致的性能。

下面的例子通过计时来展示了延后执行的效果。可以看到，当`y=...`返回的时候并没有等待它真的被计算完。


In [2]:
from mxnet import nd 
from time import time 
start=time() 
x=nd.random_uniform(shape=(2000,2000))
y=nd.dot(x,x) 
print('workloads are queued : \t %f sec ' %(time()-start)) 
print(y) 
print('workloads are queued : \t %f sec ' %(time()-start)) 


workloads are queued : 	 0.001086 sec 

[[501.1584  508.29724 495.65237 ... 492.84705 492.69092 490.0481 ]
 [508.81058 507.1822  495.1743  ... 503.10526 497.29315 493.67917]
 [489.56598 499.47015 490.17722 ... 490.99945 488.05008 483.28836]
 ...
 [484.0019  495.7179  479.92142 ... 493.69952 478.89194 487.2074 ]
 [499.64932 507.65094 497.5938  ... 493.0474  500.74512 495.82712]
 [516.0143  519.1715  506.354   ... 510.08878 496.35608 495.42523]]
<NDArray 2000x2000 @cpu(0)>
workloads are queued : 	 0.152164 sec 



延迟执行大部分情况是对用户透明的。因为除非我们需要打印或者保存结果外，我们基本不需要关心目前是不是结果在内存里面已经计算好了。

事实上，只要数据是保存在NDArray里，而且使用MXNet提供的运算子，后端将默认使用延迟执行来获取最大的性能。

## 立即获取结果

除了前面介绍的`print`外，我们还有别的方法可以让前端线程等待直到结果完成。我们可以使用`nd.NDArray.wait_to_read()`等待直到特定结果完成，或者`nd.waitall()`等待所有前面结果完成。后者是测试性能常用方法。


In [3]:
start=time() 
y=nd.dot(x,x) 
y.wait_to_read() 
time()-start 

0.10746622085571289

In [4]:
start=time() 
y=nd.dot(x,x) 
z=nd.dot(x,x) 
nd.waitall() 
time()-start 

0.11718463897705078


## 延迟执行带来的便利

下面例子中，我们不断的对`y`进行赋值。如果每次我们需要等到`y`的值，那么我们必须要要计算它。而在延迟执行里，系统有可能省略掉一些执行。


In [5]:
start=time() 
for i in range(1000):
    y=x+1 
    y.wait_to_read() 
print('No lazy evaluation : %f sec '%(time()-start))

start=time() 
for i in range(1000):
    y=x+1 
nd.waitall() 
print('No lazy evaluation : %f sec '%(time()-start))


No lazy evaluation : 0.467245 sec 
No lazy evaluation : 0.194332 sec 



## 延迟执行带来的影响

在延迟执行里，只要最终结果是一致的，系统可能使用跟代码不一样的顺序来执行，例如假设我们写


In [6]:
a = 1
b = 2
a + b

3


第一句和第二句之间没有依赖，所以把`b=2`提前到`a=1`前执行也是可以的。但这样可能会导致内存使用的变化。

下面我们列举几个在训练和预测中常见的现象。一般每个批量我们都会评测一下，例如计算损失或者精度，其中会用到`asscalar`或者`asnumpy`。这样我们会每次仅仅将一个批量的任务放进后端系统执行。但如果我们去掉这些同步函数，会导致我们将大量的批量任务同时放进系统，从而可能导致系统占用过多资源。

为了演示这种情况，我们定义一个数据获取函数，它会打印什么数据是什么时候被请求的。

第一句和第二句之间没有依赖，所以把`b=2`提前到`a=1`前执行也是可以的。但这样可能会导致内存使用的变化。

下面我们列举几个在训练和预测中常见的现象。一般每个批量我们都会评测一下，例如计算损失或者精度，其中会用到`asscalar`或者`asnumpy`。这样我们会每次仅仅将一个批量的任务放进后端系统执行。但如果我们去掉这些同步函数，会导致我们将大量的批量任务同时放进系统，从而可能导致系统占用过多资源。

为了演示这种情况，我们定义一个数据获取函数，它会打印什么数据是什么时候被请求的。


In [7]:
def get_data():
    start=time() 
    batch_size=1024 
    for i in range(60):
        if i%10==0:
            print("batch %d,time %f sec "%(i,time()-start))
        x=nd.zeros((batch_size,1024))
        y=nd.zeros((batch_size,)) 
        yield x,y 

使用两层网络和L2损失函数作为样例 

In [8]:
from mxnet import gluon 
from mxnet.gluon import nn
net=nn.Sequential() 
with net.name_scope():
    net.add(
        nn.Dense(1024),
        nn.Activation('relu'),
        nn.Dense(1024),
    )
net.initialize()
trainer=gluon.Trainer(net.collect_params(),'sgd',{})
loss=gluon.loss.L2Loss() 

我们定义辅助函数来检测内存的使用(只能在Linux中使用 ) 

In [9]:
import os 
import subprocess 
def get_mem():
    ''' get memeory usage in MB '''
    res=subprocess.check_output(['ps','u','-p',str(os.getpid())])
    return int(str(res).split()[15])/1e3

In [10]:
mem=get_mem() 
for x,y in get_data():
    loss(y,net(x)).wait_to_read()
nd.waitall() 
print('Increased memeory %f MB'%(get_mem()-mem)) 

batch 0,time 0.000008 sec 
batch 10,time 0.223729 sec 
batch 20,time 0.341261 sec 
batch 30,time 0.457223 sec 
batch 40,time 0.573264 sec 
batch 50,time 0.689856 sec 
Increased memeory 6.720000 MB


In [11]:
mem=get_mem() 
for x,y in get_data():
    loss(y,net(x)) 
print('Increased memeory %f MB'%(get_mem()-mem)) 

batch 0,time 0.000009 sec 
batch 10,time 0.024897 sec 
batch 20,time 0.044075 sec 
batch 30,time 0.059085 sec 
batch 40,time 0.074072 sec 
batch 50,time 0.087178 sec 
Increased memeory 257.924000 MB



同样对于训练，如果我们每次计算损失，那么就加入了同步

In [12]:
# from mxnet import autograd 
# mem=get_mem() 
# total_loss=0 
# for x,y in get_data():
#     print(x.shape,y.shape)
#     with autograd.record():
#         x=net(x)
#         print(x.shape) 
#         L=loss(y,x)  
#     L.backward()
#     trainer.step(x.shape[0]) 
# nd.waitall()
# print('Increased memeory %f MB'%(get_mem()-mem)) 

In [13]:
# from mxnet import autograd 
# mem=get_mem() 
# total_loss=0 
# for x,y in get_data():
#     with autograd.record():
#         L=loss(y,net(x)) 
#     total_loss+=L.sum().asscalar()
#     L.backward()
#     trainer.step(x.shape[0]) 
# nd.waitall()
# print('Increased memeory %f MB'%(get_mem()-mem)) 


batch 0,time 0.000011 sec 


MXNetError: Traceback (most recent call last):
  File "src/ndarray/ndarray.cc", line 269
NDArray.Reshape: Check failed: shape_.Size() == shape.Size() (1048576 vs. 1024) : target shape must have the same size as current shape when recording with autograd.